### Import Modules, Change Directory, Set Variables

In [0]:
output_dir = 'data/filtered data/' # CHANGE: output folder location (autoassign if none)
window = 3 # #CHANGE: assessment through which you want all features
output_name = None #CHANGE: output tsv file name (auto assign if none)

In [0]:
import os
from zipfile import ZipFile
import pandas as pd
import json
from datetime import date
output_dir = output_dir or 'jupyter/jowilder_data'
output_name = output_name or f'LAKELAND_through_window_{window}_{date.today().strftime("%Y%m%d")}.tsv'
outpath = os.path.join(output_dir,output_name)
print(f'outpath: {outpath}')

outpath: data/filtered data/LAKELAND_through_window_3_20200115.tsv


### Look to see what proc_zips are available, choose one to open

In [0]:
data_dir = 'data/input'
files = [x for x in sorted(os.listdir(data_dir),reverse=True) if 'proc.zip' in x]
get_file_by_n = lambda n: os.path.join(data_dir, files[n])
[f'{i}: {f}' for i,f in enumerate(files)][:10]

FileNotFoundError: ignored

In [0]:
read_file = 0 # CHANGE: This is the enum of the file you wish to open

### Import to dataframe, verify roughly that is it what is expected

In [0]:
zf = ZipFile(get_file_by_n(read_file))
with zf.open(zf.namelist()[0]) as f:
    df = pd.read_csv(f,index_col='sessID')
df.head()

,lvl0_EventCount,lvl1_EventCount,lvl2_EventCount,lvl3_EventCount,lvl4_EventCount,lvl5_EventCount,lvl6_EventCount,lvl7_EventCount,lvl8_EventCount,lvl9_EventCount,...,sess_ActiveEventCount,continue,language,audio,fullscreen,version,debug,num_play,persistentSessionID,sessDuration
sessID,,,,,,,,,,,,,,,,,,,,,
19110002150060536,133,181.0,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,197,0,english,1,0,15,0,1,19110002150060536,705.210
19110007194177108,91,172.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,149,0,english,1,0,15,0,1,19110007194177108,674.433
19110007194177108,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50,1,english,1,0,15,0,2,19110007194177108,246.690
19110008292229944,118,183.0,156.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,150,0,english,1,0,15,0,1,19110008292229944,857.575
19110008450097140,272,246.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,65,1,english,1,0,15,0,1,19100309270560400,677.492


In [0]:
df.shape

(17116, 1638)

### Import Schema, look at available features

In [0]:
schema_path = 'data/input/LAKELAND.json'
with open(schema_path) as f:
    schema = json.load(f)
features = schema['features']
lvl_feature = lambda n,f: f'lvl{n}_{f}'

In [0]:
print(*[[k,list(v.keys())] for k,v in features.items()],sep='\n\n')

['perlevel', ['EventCount', 'ActiveEventCount', 'durationInSecs', 'count_gamestate_logs', 'count_achievements', 'count_buy_home', 'count_buy_food', 'count_buy_farm', 'count_buy_fertilizer', 'count_buy_livestock', 'count_buy_skimmer', 'count_buy_sign', 'count_buy_road', 'money_spent', 'money_spent_home', 'money_spent_food', 'money_spent_farm', 'money_spent_fertilizer', 'money_spent_livestock', 'money_spent_skimmer', 'money_spent_sign', 'money_spent_road', 'min_num_home_in_play', 'max_num_home_in_play', 'min_num_food_in_play', 'max_num_food_in_play', 'min_num_farm_in_play', 'max_num_farm_in_play', 'min_num_fertilizer_in_play', 'max_num_fertilizer_in_play', 'min_num_livestock_in_play', 'max_num_livestock_in_play', 'min_num_skimmer_in_play', 'max_num_skimmer_in_play', 'min_num_sign_in_play', 'max_num_sign_in_play', 'min_num_road_in_play', 'max_num_road_in_play', 'min_num_water_in_play', 'max_num_water_in_play', 'min_num_poop_in_play', 'max_num_poop_in_play', 'min_num_milk_in_play', 'max_nu

### Drop columns past given assessment, drop rows that did not answer the last question

In [0]:
def features_through_window_n(n):
    ret = []
    for lvl in range(n):
        for f in features['perlevel']:
            ret.append(lvl_feature(lvl,f))
    for f in features['per_custom_count']:
        pass
    ret.extend([f for f in features['aggregate'] if f != 'sessID'])
    return ret
assessment_df = df[features_through_window_n(window)]
# assessment_df = assessment_df.dropna(subset=last_q_subset, how='all')

In [0]:
assessment_df.shape

(17116, 651)

In [0]:
assessment_df.head()

,lvl0_EventCount,lvl0_ActiveEventCount,lvl0_durationInSecs,lvl0_count_gamestate_logs,lvl0_count_achievements,lvl0_count_buy_home,lvl0_count_buy_food,lvl0_count_buy_farm,lvl0_count_buy_fertilizer,lvl0_count_buy_livestock,...,sess_ActiveEventCount,continue,language,audio,fullscreen,version,debug,num_play,persistentSessionID,sessDuration
sessID,,,,,,,,,,,,,,,,,,,,,
19110002150060536,133,77.0,NaN,4.0,2.0,1.0,1.0,1.0,1.0,NaN,...,197,0,english,1,0,15,0,1,19110002150060536,705.210
19110007194177108,91,51.0,NaN,3.0,2.0,1.0,1.0,1.0,NaN,NaN,...,149,0,english,1,0,15,0,1,19110007194177108,674.433
19110007194177108,184,50.0,NaN,10.0,5.0,2.0,4.0,NaN,NaN,NaN,...,50,1,english,1,0,15,0,2,19110007194177108,246.690
19110008292229944,118,37.0,NaN,6.0,2.0,1.0,1.0,2.0,1.0,1.0,...,150,0,english,1,0,15,0,1,19110008292229944,857.575
19110008450097140,272,42.0,NaN,7.0,4.0,NaN,NaN,1.0,4.0,1.0,...,65,1,english,1,0,15,0,1,19100309270560400,677.492


### Export

In [0]:
### Import to dataframe, verify roughly that is it what is expected
assessment_df.to_csv(outpath, sep='\t')